In [42]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [43]:
import boto3

In [44]:
def SessionManager(profile="default", region="eu-central-1"):
    return boto3.Session(
        profile_name=profile,
        region_name=region
    )

In [45]:
import logging 
import os
from datetime import datetime

In [46]:
logging.basicConfig(
    format= " [%(asctime)s ] %(lineno)s - %(levelname)s - %(message)s",
    level = logging.INFO
)

In [47]:
from botocore.exceptions import ClientError

class AwsException(Exception):
    def __init__(self, error_message: str , error_details:ClientError):
        super().__init__(error_message)

        self.error_message = error_message
        self.error_details = error_details

        # extract AWS errors
        if isinstance(error_details, ClientError):
            error_info = error_details.response.get("Error", {})
            self.error_code = error_info.get("Code")
            self.aws_message = error_info.get("Message")
            self.request_id = error_details.response.get("ResponseMetadata", {}).get("RequestId")
            self.operation_name = error_details.operation_name
        else:
            self.error_code = None
            self.aws_message = None
            self.request_id = None
            self.operation_name = None

    def __str__(self):
        return (
            f"{self.error_message} | "
            f"AWS Code: {self.error_code} |"
            f"AWS Message: {self.aws_message} |"
            f"Operation: {self.operation_name} |"
            f"RequestID: {self.request_id}"            
        )

## PAGINATOR

In [48]:
class Paginator:

    @staticmethod
    def aws_paginator(client,method,key,**kwargs):
        try:
            paginator = client.get_paginator(method)
            print (f"Paginator: {paginator}")
            for page in paginator.paginate(**kwargs):
                for item in page.get(key, []):
                    yield item

        except ClientError as e:
            raise AwsException("AWS pagination failed", e)
        
        except Exception as e:
            raise AwsException("unexpected pagination error", e)


In [ ]:
from abc import ABC , abstractmethod

class BaseCollector(ABC):
    def __init__(self,session,region):
        self.session = session
        self.region = region

    @abstractmethod
    def collect(self):
        pass
    def handle_aws_error(self,message, error):
        raise AwsException(message, error)



## EC2 Collector

In [ ]:
class EC2Collector(BaseCollector):

    def collect(self):
        logging.info(f"Starting the EC2 collection in region: {self.region}")

        ec2 = self.session.client("ec2", region_name=self.region)
        inventory = []

        try:
            for reservation in Paginator.aws_paginator(
                ec2,
                "describe_instances",
                "Reservations"
            ):
                for instance in reservation.get("Instances" , []):

                    tags = {
                        t.get("key"): t.get("Value")
                        for t in instance.get("Tags", [])
                    }

                    inventory.append({
                        "service": "ec2",
                        "instance_id": instance.get("InstanceId"),
                        "state": instance.get("State", {}).get("Name"),
                        "region": self.region,
                        "tags": tags
                    })
            logging.info(f"Collected {len(inventory)} EC2 instances")
            return inventory
        except ClientError as e:
            logging.error("EC2 collection failed", exc_info=True)
            self.handle_aws_error("EC2 collection failed", e)


 [2026-02-22 09:10:30,330 ] 1392 - INFO - Found credentials in shared credentials file: ~/.aws/credentials


Starting EC2 collection: ap-south-1
Paginator: <botocore.client.EC2.Paginator.DescribeInstances object at 0x000002B091AED000>


In [ ]:
session = SessionManager()
ec2_data = EC2Collector(session, "eu-central-1").collect()